# Exercise session 12: Spiking Neural Networks

Welcome to the twelth exercise sessoin of the itnro to neuroinformatics course. In this notebook, we will investigate some aspects of spiking neural networks (SNNs). In particular, we first discuss the BCM learning rule, which can be considered an extension of the Hebbian learning rule introduced in lecture 10. Next, we discuss two simple SNNs applications inspired by biology: In part 2 of the exercise, we discuss interaural time difference (ITD) in the case of sound locations observed by the owl. In part 3, we discuss adaptation in vision, which is a mechanism that allows the visual system to adjust its sensitivity to the input.

---

# Table of contents

* [Packages](#packages)
* [Introduction](#0)
* [1: BCM Learning rule](#1)
  * [1.1: Math](#1.1)
  * [1.2: Weight decay](#1.2)
  * [1.3: Simulation](#1.3)
* [2: Sound localization with the Jeffress model](#2)
  * [2.1: Jeffress model](#2.1)
  * [2.2: Exploring the Jeffress model](#2.2)
* [3: Adaptation in vision](#3)
  * [3.1: Leaky integrate-and-fire neuron](#3.1)
  * [3.2: Interactive exploration](#3.2)
  * [3.3: Adaptation in neurons](#3.3)
  * [3.4: Modeling](#3.4)

  ---

# Packages <a name="packages"></a>

First, we need to update the widgets since we use a never version than the Colab default.

In [1]:
!pip install -U ipywidgets
!pip install brian2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.4 MB/s eta 0:00:00


Next, we download additional files required by this tutorial.

In [2]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
print(f"IN_COLAB: {IN_COLAB}")

IN_COLAB: True


In [3]:
if IN_COLAB:
    # Download util scripts
    !mkdir utils_ex12
    !wget -P utils_ex12/ https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex12/bcm.py
    !wget -P utils_ex12/ https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex12/adaptation.py
    !wget -P utils_ex12/ https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex12/jeffress.py

--2025-12-03 21:25:40--  https://github.com/ManteLab/Iton_notebooks_public/raw/refs/heads/main/utils_ex12/bcm.py
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ManteLab/Iton_notebooks_public/refs/heads/main/utils_ex12/bcm.py [following]
--2025-12-03 21:25:41--  https://raw.githubusercontent.com/ManteLab/Iton_notebooks_public/refs/heads/main/utils_ex12/bcm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9058 (8.8K) [text/plain]
Saving to: ‘utils_ex12/bcm.py’

bcm.py              100%[===================>]   8.85K  --.-KB/s    in 0s      

2025-12-03 21:25:41 (72.3 MB/s) - ‘utils_ex12/bcm.py’ 

In [4]:
if IN_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

In [5]:
from utils_ex12.bcm import *
from utils_ex12.adaptation import *
from utils_ex12.jeffress import *

---

# Introduction <a name="0"></a>

Artificial Neural Networks (ANNs, see lab from last week), as used in many machine learning applications today, are incredibly powerful tools. However, their learning algorithm backpropagation of error is not biologically plausible and comes with many other downsides like the susceptibility to adversarial attacks, need for large amounts of data, etc.

In this lab, we explore Spiking Neural Networks (SNNs), a more biologically plausible model of computation that uses *spikes* to transmit information. Unlike ANNs, SNNs operate in a discrete-time or event-driven paradigm, where spikes occur at specific time steps rather than continuously.

#### Learning in Spiking Neural Networks

While it is theoretically possible to train SNNs using techniques like backpropagation and surrogate gradients (a concept we will cover in a future lab), this session focuses on **local learning rules**, which are inspired by how biological synapses adapt. Local learning rules:

- Do not rely on global error signals.
- Use local information, such as pre- and post-synaptic activity, to modify synaptic weights.
- Include biologically inspired mechanisms like Spike-Timing Dependent Plasticity (STDP, see exercise 10).

---

# 1: BCM Learning Rule <a name="1"></a>


The BCM theory, developed by Bienenstock, Cooper, and Munro (hence the name) in 1982, proposes a model of synaptic modification that accounts for experimental observations of neuronal selectivity in the primary sensory cortex and its dependency on input patterns. This theory describes how synaptic changes occur in a biologically plausible manner by linking synaptic plasticity to both presynaptic and postsynaptic activity.

Key concepts of this theory includes:

- **Hebbian-like Learning:** Synaptic changes depend on the interaction between the presynaptic activity and a *nonlinear function* of the postsynaptic activity.
- **Sliding Modification Threshold:** The model introduces a dynamically varying threshold, $\theta_M$, which governs whether synaptic change is potentiation (strengthening) or depression (weakening). This threshold adjusts based on the previous activity of the postsynaptic neuron.
- **Activity Dependency:** The learning rule $\phi(y; \theta_M)$, which determines synaptic change, takes the postsynaptic activity $y$ and compares it to the threshold $\theta_M$:
  - When $y < \theta_M$: The function is negative, leading to synaptic weakening (depression).
  - When $y > \theta_M$: The function is positive, resulting in synaptic strengthening (potentiation).

This sliding threshold provides stability to the learning process by enabling competition between incoming patterns rather than relying on converging afferents. This mechanism allows the model to dynamically adapt to changing input statistics and maintain balance in synaptic strengths.

<p align="center">
  <img src="https://raw.githubusercontent.com/ManteLab/Iton_notebooks_public/refs/heads/main/utils_ex12/Figures/BCM_rule.png" alt="The BCM Synaptic Modification Rule" width="400">
</p>
<p align="center">
  <em>The BCM Synaptic Modification Rule. $y$ denotes the output activity of the neuron, $\theta_M$ is the modification threshold. (Image is from Scholarpedia)</em>
</p>

---




## 1.1: Math <a name="1.1"></a>

<p align="center">
  <img src="https://www.fabriziomusacchio.com/assets/images/posts/nest/bcm_scheme.png" alt="A neuron with multiple inputs" width="400">
</p>
<p align="center">
  <em>A single neurons with multiple inputs and one output (Image is from https://www.fabriziomusacchio.com)</em>
</p>

Lets define $x_i$ as the presynaptic activity of the $i$-th neuron, $y$ as the postsynaptic activity, and $w_i$ as the synaptic weight from $x_i$ to $y$.

The change in synaptic weight $dw_i/dt$ is given by:

$$
    \frac{dw_i}{dt} = \eta \cdot x_i \cdot y \cdot (y - \theta_M)
$$

Where:
- $\eta$: Learning rate.
- $x_i$: Presynaptic activity of neuron $i$.
- $y$: Postsynaptic activity.
- $\theta_M$: The modification threshold.

The modification threshold $\theta_M$ is a function of the average postsynaptic activity $\langle y \rangle$:

$$
    \theta_M = \phi \cdot \langle y \rangle^p, \text{, with } p > 1
$$

Here, $\phi$ is a scaling factor hat adjusts the sensitivity of $\theta_M$ to $\langle y \rangle$. and $p$ is an power factor to ensure a super-linear relationship.
The choice of $p>1$ ensures that the threshold $\theta_M$ grows faster than $\langle y \rangle$. This has important implications:

1. **Activity-Dependent Stability:** High postsynaptic activity leads to a rapid increase in $\theta_M$, which in turn makes it harder for future activity to exceed $\theta_M$. This prevents runaway potentiation, where synapses become excessively strong.
2. **Adaptive Plasticity:** Lower activity causes $\theta_M$ to drop more gradually, allowing for synaptic depression to occur more frequently in underactive synapses.

To provide a complete mathematical model, we consider the dynamics of the time-averaged postsynaptic activity $\langle y \rangle$ which evolves according to:

$$
    \tau \frac{d\langle y \rangle}{dt} =  - \langle y \rangle + y
$$

This equation describes the dynamics of $\langle y \rangle$.

-  $-\langle y \rangle$: This term causes  $\langle y \rangle$ to decay toward zero over time if there is no input ($y = 0$).
-  $+y$: This term introduces the influence of the instantaneous activity  $y$ on  $\langle y \rangle$.
-  $\frac{d\langle y \rangle}{dt}$: The rate of change of the time-averaged activity.
-  $\tau$: The time constant, which controls how quickly the time-averaged activity  $\langle y \rangle$ adjusts to changes in  $y$. Larger  $\tau$ values mean slower adjustments, while smaller  $\tau$ values allow for faster adaptation.


##### Intuition Behind the Formula

The threshold $\theta_M$ serves as a self-regulating mechanism for the neuron, ensuring that the balance between potentiation and depression adapts to the neuron's firing rate.
- For example:
  - If $\langle y \rangle$ (average activity) is **low**, $\theta_M$ will also be low, making it easier for postsynaptic activity $y$ to exceed $\theta_M$, resulting in potentiation.
  - If $\langle y \rangle$ is **high**, $\theta_M$ rises sharply due to the super-linear term ($\langle y \rangle^p$), leading to more synaptic depression.

This adaptive property allows neurons to stabilize their synaptic strengths while remaining sensitive to changes in activity patterns, a key feature of biologically plausible learning mechanisms.

---


## 1.2: Weight Decay <a name="1.2"></a>

In the literature, various extensions and refinements of the BCM rule have been proposed, incorporating additional factors such as metaplasticity, homeostatic regulation, and network interactions. These modifications aim to capture the complex interplay of factors influencing synaptic plasticity in neural circuits.

A variant of the BCM rule adds a weight decay term $-\epsilon w_i$ to the Hebbian-like term $y \cdot (y - \theta_M) \cdot x_i$, instead of multiplying it by the learning rate, allowing for a more nuanced regulation of synaptic changes over time:

$$
\frac{dw_i}{dt} = y \cdot (y - \theta_M) \cdot x_i - \epsilon w_i
$$

The weight decay term acts to stabilize the weight, preventing it from growing indefinitely. This modified BCM rule is different from the previous BCM formulation in that it explicitly includes a mechanism to reduce the synaptic weight over time, proportional to its current value. This is a common modification in neural network models to ensure weights do not grow without bound and to introduce a form of weight regularization.

---


> **Assignment 1**
>
> Consider a network where most neurons fire frequently ($y$ is consistently high) versus a network where only a few neurons fire occasionally ($y$ is consistently low). What happens to the sliding threshold $\theta_M$ in both cases and how does this affect synaptic potentiation and depression?  

**Solution:**




> **Assignment 2**
>
> Analyze the behavior of the synapse when $\theta_M$ is (1.) very high compared to typical postsynaptic activity ($\theta_M \gg y$) and (2.) Very low compared to typical postsynaptic activity ($\theta_M \ll y$). What happens to the balance between potentiation and depression in each case? How does this affect the synaptic weights?

**Solution:**



> **Assignment 3**
>
> The modified BCM rule introduces a weight decay term $-\epsilon w_i$. Think about a scenario where $\epsilon = 0$ (no decay). What happens to the synaptic weights over time?  How does adding weight decay contribute to network stability?  

**Solution:**

---

## 1.3 Simulation <a name="1.3"></a>

In the following interactive simulation, you can explore the behavior of the BCM learning rule. We use two input neurons $x_1$ and $x_2$ that generate random input patterns. The synaptic weights $w_1$ and $w_2$ are modified according to the BCM rule based on the postsynaptic activity $y$ and the sliding threshold $\theta_M$.

You can adjust the following parameters:

- **Frequency Input 1/2**: Frequency of the input signals (pulses).
- **Amplitude Input 1/2**: Amplitude of the input signals.
- **Phase Shift**: Phase shift between the input signals.
- **Use Weight Decay:** Whether to include weight decay in the learning rule.

Optionally, you can also adjust the following parameters in the second tab. However, we recommend keeping the default values:
- **Learning Rate ($\eta$):** Determines the speed of synaptic changes.
- **Time Constant ($\tau$):** Time constant for averaging postsynaptic activity.
- **Decay Rate ($\epsilon$):** Rate of weight decay.
- **Simulation Time:** Total simulation time in ms.

As output, you see the following plots:
- **Presynaptic Inputs:** Input signals generated by the two neurons.
- **Postsynaptic Activity and Sliding Threshold:** Postsynaptic activity $y$ and the sliding threshold $\theta_M$.
- **Evolution of Synaptic Weights:** Synaptic weights $w_1$ and $w_2$ over time.
- **Cumulative Inputs:** Cumulative effect of inputs over time.


Note that we limit the weights to the range $[0, 1]$ to ensure they remain within a reasonable range. Without weight decay, the weights can grow without bound, leading to instability (*Hint: This information might be useful to answer the assignments above...*).


In [6]:
iplot_bcm_model()

---

> **Assignment 4**
>
> Use the default settings (execute the cell above to reset to default settings). Explain why the postsynaptic activity increases over time.  Why does it start at $0.4$ and reach $0.8$ by the end? Provide a detailed explanation using the relevant formulae.

**Solution:**





> **Assignment 5**
>  
> Use the default settings (reset the parameters by executing the cell above). Initially, the synaptic weights increase almost linearly until they reach the upper limit of $1$ (where the weights are clipped). Next, increase the frequency of Input 1 to $8 \, \text{Hz}$ and observe the behavior (press update). Notice that the weight $w_1$ grows incrementally, increasing only when Input 1 ($x_1$) is active. Explain why the growth of $w_2$ becomes nonlinear, even though Input 2 ($x_2$)) remains unchanged.

**Solution:**



> **Assignment 6**
>
> Use the default settings (reset the parameters by executing the cell above). Change the amplitude of Input 2 ($x_2$) to  $0.1$ and observe the results (press update). You will notice that $w_1$ grows faster than $w_2$. Then, enable weight decay (press update). What happens to  $w_2$, and why does this behavior occur?

**Solution:**



> **Assignment 7**
>
> Use the default settings (reset the parameters by executing the cell above). Modify the input settings as follows: set the frequency of Input 2 ( $x_2 $) to  $5 \, \text{Hz}$, the amplitude of Input 1 ($x_1$) to  $1.0$, and the amplitude of Input 2 ( $x_2$) to  $0.1$. Observe the evolution of the synaptic weight $w_1$ over time and describe how the postsynaptic activity changes.

**Solution:**

---

# 2: Sound Localization with the Jeffress model <a name="2"></a>

Most animals have two ears.
This allows the perception of different arrivals of a sound in time, called interaural time difference (ITD).
The [Jeffress model](http://www.scholarpedia.org/article/Jeffress_model) uses the interaural time difference to estimate the direction of sound sources (in 2-dimensions).

---



## 2.1: Jeffress model <a name="2.1"></a>

The Jeffress model is a neural model that explains how the brain can locate the source of a sound in space using ITDs. Imagine an owl perched on a branch, listening for the scurrying of a mouse below. When the mouse makes a sound, the sound waves reach the owl's two ears at slightly different times. This tiny difference in arrival time, the ITD, is the key to figuring out where the mouse is.

Here's how the Jeffress model works:

* **Delay Lines:** Think of the owl's auditory system as having a set of "delay lines" that transmit the sound signals from each ear to a set of "coincidence detector" neurons in the brain.  Each delay line has a slightly different length, introducing a specific time delay.

* **Coincidence Detectors:**  These neurons act like tiny logic gates. They only fire when signals from *both* ears arrive at the *same time*.

* **Sound Localization:** Imagine the mouse directly before the owl. The sound reaches both ears simultaneously, so the signal travels down the delay lines and activates a coincidence detector in the middle. If the mouse moves to the right, the sound reaches the right ear first. The signal from the right ear has to travel further down the delay lines to find a coincidence detector that also receives the (delayed) signal from the left ear. The location of the activated coincidence detector tells the owl the direction of the sound!

[This](https://www.st-andrews.ac.uk/~wjh/pred_prey/jeffress/jeffress.html) shows a nice animation of this effect.

To create these "delay lines," we strategically connect neurons between the left and right ears, introducing varying delays in the connections. This arrangement ensures that signals from each ear arrive at different coincidence detector neurons with specific time offsets, creating a "temporal map" of the sound's location.

The illustration below shows how this is achieved using the Jeffress model network. Notice how the input from the right ear is connected in a reversed order compared to the input from the left ear. This clever wiring creates the delay lines that are crucial for sound localization.

![](https://raw.githubusercontent.com/ManteLab/Iton_notebooks_public/refs/heads/main/utils_ex12/Figures/jeffress_network.png)

The next image provides a complete picture of the concept. You can see the sound source (the mouse), the owl with its two ears, and the Jeffress model network. In this scenario, a coincidence detector on the right side is activated. This happens because the sound reaches the left ear first, allowing its signal to travel further down the delay lines and meet the signal from the right ear at that specific neuron. This activation pattern tells the owl that the sound is coming from the left.

![](https://raw.githubusercontent.com/ManteLab/Iton_notebooks_public/refs/heads/main/utils_ex12/Figures/concept.png)



## 2.2: Exploring the Jeffress Model <a name="2.2"></a>

The code below simulates the Jeffress model. You can interactively explore how the model responds to different sound source locations and neuron counts using the following sliders:

* **`Angle (deg)`:**  Adjust the angle from which the sound arrives. The green arrow in the visualization indicates the sound source direction.
* **`# neurons`:** Change the number of neurons in the Jeffress model.  These neurons are visually represented around the owl, each corresponding to a specific sound location. Increasing the number of neurons should improve the accuracy of sound localization.

In [7]:
jeffress_app()

---

> **Assignment 8a**
>
> Keeping `# neurons = 30` while changing the `Angle (deg)` to `0` degrees, `90` degrees, and `180` degrees. What do we observe?

**Solution:**


> **Assignment 8b**
>
> What does the `input delay` plot show? Investigate with the angles from 8a.

**Solution:**

> **Assignment 9**
>
> Reduce the `# neurons` and investigate with different angles. What do we observe?

**Solution:**`

> **Assignment 10**
>
> The Jeffress model concerns sound localization in two dimensions. What are its limits? To investigate this, switch to the `Investigate limits` tab, where we can input all possible sound sources in two dimensions. Investigate the model, for example, for the angles of `90` degrees and `270` degrees. What do we observe?

**Solution:**



> **Assignment 11**
>
> Can you summarize the limits we found in the last assignment more generally? Think about symmetries in the owl's input.

**Solution:**

> **Assignment 12**
>
> How can this limitation be solved?

**Solution:**

---

# 3: Adaptation in Vision <a name="3"></a>

Visual adaptation refers to the ability of the visual system to adjust its sensitivity to changes in light or stimulus intensity. This process allows the eyes to maintain optimal perception under varying lighting conditions, enhancing contrast detection and reducing visual overload. In neural terms, visual adaptation can be modeled as a gradual change in the response of a neuron to stimuli, where repeated or prolonged exposure to a stimulus leads to a reduction in its response. This mechanism is essential for perceiving the environment in a way that remains stable despite variations in input.

In the context of a neuron model, visual adaptation can be incorporated by modifying the neuron's firing behavior in response to repeated stimuli. The neuron gradually becomes less sensitive to constant or repetitive inputs, similar to how visual sensitivity decreases under constant light exposure. This adaptation can be modeled by adding an adaptation current to the standard leaky integrate-and-fire (LIF) model, where the current gradually builds with each spike, reducing the neuron's firing rate over time.

In the following, we try to understand this adaptation mechanism. Therefore, we first discuss the standard LIF model before extending it with an adaptation term.

---

## 3.1: Leaky Integrate-and-Fire Neuron <a name="3.1"></a>

Before we discuss adaptation, we will introduce the Leaky Integrate-and-Fire (LIF) neuron model. The LIF neuron is a simple model that describes the dynamics of a neuron's membrane potential. The model consists of two main components:

### Mathematical Description of the LIF Model
The LIF neuron is governed by the following dynamics:

1. **Membrane Potential Dynamics**:
   The change in membrane potential $V(t)$ is given by:

$$
   \tau_m \frac{dV(t)}{dt} = -(V(t) - V_L) + \frac{I(t)}{g_L}
$$

   Where:
   - $\tau_m$: Membrane time constant [ms].
   - $V_L$: Leak reversal potential [mV].
   - $g_L$: Leak conductance [nS].
   - $I(t)$: Input current [pA].


The equation models how the voltage across a neuron's membrane, $V(t)$, changes over time.  The left side of the term $\tau_m \frac{dV(t)}{dt}$
term represents how quickly the membrane potential $V(t)$ changes over time. The speed of this change depends on the **membrane time constant**, $\tau_m$, which controls how fast the neuron responds to inputs. A larger $\tau_m$ means the neuron reacts more slowly, smoothing out rapid changes, while a smaller $\tau_m$ allows the neuron to respond more quickly.

The term on the right side $-(V(t) - V_L) + \frac{I(t)}{g_L}$ describes the two main forces acting on $V(t)$:

- **Leak Term: $-(V(t) - V_L)$**: This term pulls the membrane potential back toward a steady "resting value," $V_L$.
   - If $V(t)$ is above $V_L$, the term is negative, pushing $V(t)$ down.
   - If $V(t)$ is below $V_L$, the term is positive, pulling $V(t)$ up.

- **Input Current: $\frac{I(t)}{g_L}$**:  This term represents external stimulation to the neuron in the form of an input current, $I(t)$, divided by the leak conductance, $g_L$.
   - $I(t)$: Describes how much external energy is pushing the membrane potential up.
   - $g_L$: Dampens the effect of $I(t)$ based on the neuron's "leakiness" (how easily it allows ions to flow).

Thus, very simply put, the first part of the equation tries to keep the membrane potential near a stable resting value $V_L$ while the second part pushes it up or down based on the input current.


2. **Spike Generation**:
   - When $V(t) \geq V_{th}$ (threshold potential), the neuron "spikes," and:
     - $V(t)$ is reset to $V_{reset}$.
     - A refractory period $t_{ref}$ prevents spiking immediately after.


3. **Reset Dynamics**:
   During the refractory period, $V(t)$ is clamped to $V_{reset}$.




---

> **Assignment 13**
>
> Assume you have these parameters given: $V_L = -70mV$ and $V(t) = -60mV$, while the input current is almost $0$. What happens with the membrane potential?

**Solution:**



> **Assignment 14**
>
> Identical to assignment 13, the parameters $V_L = -70mV$ and $V(t) = -60mV$ are given but the input current is very high. What happens with the membrane potential?

**Solution:**



> **Assignment 15**
>
> Assume you make a measurement of the spike time. The initial membrane potential is equal to the reset potential. Then you apply a constant input current. You measure a spike after $5$ms before the membrane potential is reset. When you keep the input current constant - When will the next spike occur according to the LIF model?

**Solution:**


---

## 3.2: Interactive Exploration <a name="3.2"></a>

Modify the parameters below to observe their impact on the behavior of the LIF neuron:

- **Input current (`I`)**: Adjust the strength of the stimulus.
- **Threshold (`V_th`)**: Make the neuron more or less sensitive.
- **Reset potential (`V_reset`)**: Change how much the neuron "resets" after a spike.
- **Membrane time constant (`tau_m`)**: Control how quickly the neuron integrates input.
- **Refractory period (`tref`)**: Set the time the neuron is inactive after spiking.


In [8]:
simulate_lif_model()

interactive(children=(FloatSlider(value=-55.0, description='Threshold (V_th)', layout=Layout(width='500px'), m…

---

> **Assignment 16**
>
> For the LIF simulation above, adjust all parameters (threshold, reset potential, membrane time constant, input current, and refractory period) and describe what you observe.

**Solution:**

---

## 3.3: Adaptation in Neurons <a name="3.3"></a>


We now extend the LIF model by introducing an *adaptation current* $w(t)$, which modifies the membrane potential dynamics. Adaptation introduces a slow, spike-triggered feedback mechanism that reduces spiking over time, mimicking biological processes such as ion channel inactivation or hyperpolarization currents.

### Mathematical Description of the LIF Model with Adaptation

1. **Adaptation Current Dynamics**:
   The adaptation current $w(t)$ evolves according to:

   $$
   \tau_w \frac{dw(t)}{dt} = -w(t) + b \cdot S(t)
   $$

   Where:
   - $\tau_w$: Adaptation time constant [ms], controlling how slowly or quickly $w(t)$ changes.
   - $b$: Increment to the adaptation current per spike [pA], determining the strength of adaptation.
   - $S(t)$: Spike indicator function, equal to 1 when $V(t) \geq V_{th}$ (the neuron spikes) and 0 otherwise.

   This equation ensures that:
   - When the neuron spikes ($S(t) = 1$), $w(t)$ increases by $b$.
   - Between spikes ($S(t) = 0$), $w(t)$ decays back toward zero with a time constant $\tau_w$.

2. **Membrane Potential Dynamics with Adaptation**:
   The membrane potential dynamics are now influenced by $w(t)$, modifying the input current $I(t)$:

   $$
   \tau_m \frac{dV(t)}{dt} = -(V(t) - V_L) + \frac{I(t) - w(t)}{g_L}
   $$

   Compared to the standard LIF model, the term $w(t)$ subtracts from the input current $I(t)$, effectively reducing the driving force that increases $V(t)$. This creates a spike frequency adaptation effect, where spiking slows down after sustained activity.

Intuitively, the adaptation current $w(t)$ acts as a negative feedback mechanism, growing stronger after each spike ($b$) and decaying slowly over time ($\tau_w$). It also inhibits further spiking by opposing the input current $I(t)$, reducing the membrane potential increment $\frac{I(t) - w(t)}{g_L}$.

The combined dynamics of $V(t)$ and $w(t)$ ensure that initially, the neuron fires frequently in response to strong input. Over time, as $w(t)$ builds up, spiking slows down, mimicking the adaptation observed in biological neurons.


3. **Spike Generation and Reset Dynamics**:
- When $V(t) \geq V_{th}$, the neuron spikes, and:
   - $V(t)$ is reset to $V_{reset}$.
   - $w(t)$ is incremented by $b$, increasing the adaptation current.
   - A refractory period $t_{ref}$ follows, during which $V(t)$ is clamped to $V_{reset}$.





---

> **Assignment 17**
>
> In the extended LIF model, the adaptation current $w(t)$ grows larger with each spike and decays slowly over time. Explain intuitively how this mechanism affects the neuron's firing behavior. Why might this be useful for a neuron in a biological system?

**Solution:**


>**Assignment 18**
>
> The parameters $\tau_w$ (adaptation time constant) and $b$ (adaptation increment per spike) control how adaptation works.  
> - If $\tau_w$ is very small, what effect would this have on spiking behavior?  
> - If $b$ is very large, how would the neuron's firing pattern change over time?

**Solution:**

---

## 3.4: Modeling <a name="3.4"></a>

### Task Description

The adjustable parameters allow you to simulate the effects of neuronal adaptation and observe its impact on the neuron's spiking behavior and adaptation current. This can be related to how sensory systems, like the retina, adjust to changing environments (e.g., adapting from bright light to darkness).

You can adjust the following parameters:

1. **Pattern Duration (`duration_slider`)**: Controls the period of the input current's sinusoidal pattern, representing flickering light.
   - **Effect**: A smaller value increases the flicker rate, while a larger value slows it down.

2. **Input Current Factor (`input_current_slider`)**: Scales the amplitude of the input current, mimicking changes in stimulus intensity.
   - **Effect**: Larger values increase the neuron's excitation, leading to more frequent spiking.

3. **Adaptation Increment (`b_slider`)**: Adjusts the increase in adaptation current (`w`) after each spike.
   - **Effect**: Larger values enhance adaptation, reducing the firing rate after repeated spiking.

4. **Adaptation Time Constant (`tau_w_slider`)**: Determines how quickly the adaptation current (`w`) decays back to baseline after a spike.
   - **Effect**: Smaller values cause faster recovery, while larger values prolong the effects of adaptation.

Based on your settings, you can observe these plots:

1. **Input Current Over Time**:  Shows the time-varying sinusoidal input current, corresponding to a flickering light stimulus.
   - **Interpretation**: Higher peaks indicate brighter periods, while valleys represent dimmer periods.

2. **Spike Trains**: Compares spike timings of the neuron:
     - *Without Adaptation*: Spikes are driven only by the input current, without any modulation from adaptation.
     - *With Adaptation*: Spikes include the effects of adaptation, showing reduced firing rates after high activity.

3. **Adaptation Current (`w`)**:  Displays the adaptation current over time when adaptation is enabled.
   - **Interpretation**: Peaks correspond to moments when the neuron spikes, and the current builds up with sustained activity. The decay rate is determined by the adaptation time constant (`tau_w`).


By adjusting the parameters, you can simulate the neural mechanisms underlying light adaptation in the retina, where photoreceptors and downstream neurons adapt to maintain sensitivity across a wide range of illumination levels.


In [9]:
adaptation_model()

---

> **Assignment 19**
>
> Using the provided interactive simulation, adjust the four sliders to explore the impact of different parameters on the neuron's behavior. Adjust each parameter individually and observe the changes in the input current, spike trains, and adaptation current plots.

**Solution:**


---

# End of this exercise session